In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy.sparse import csc_matrix, csr_matrix, hstack
import time
import numpy as np
import pickle

from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization,CuDNNLSTM,Activation,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.utils import shuffle
from keras.preprocessing import text, sequence
from sklearn.metrics import log_loss,roc_auc_score
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
import re
from keras import backend as K
import time
import gc
# To get learning rate


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def get_nn_model(cols,doc_cols=[],numu_cols=[]):
    """
    cols, used to do ebd and dense layers
    doc_cols: used to do rnn
    there can be overlaps
    """
    input_list = []
    concat_list = []
    numu_list = []
    for col in cols:
        max_feature = len(info_dict[prefix_input_nonDoc+col]['tok'].index_word)
        embed_size = int(np.log2(max_feature)/np.log2(1.5))
        if embed_size< 2:
            embed_size = 2
        cur_input = Input(shape=(1, ),name = prefix_input_nonDoc+col)
        
       
        embed_layer = Embedding(max_feature,
                            embed_size,
                            input_length=1,
                            trainable=True,
                            embeddings_regularizer=l2(0.0005),
                            name='ebd_'+col)(cur_input)
        embed_layer = SpatialDropout1D(0.5)(embed_layer)
        x = Flatten()(embed_layer)
        input_list.append(cur_input)
        concat_list.append(x)
    for col in doc_cols:
        max_feature = len(info_dict[prefix_input_Doc+col]['tok'].index_word)
        embed_size = int(np.log2(max_feature)/np.log2(1.5))
        if embed_size< 2:
            embed_size = 2
        input_shape = sequence_size_dict[col]
        cur_input = Input(shape=(input_shape, ),name = prefix_input_Doc+col)
        embed_layer = Embedding(max_feature,
                            embed_size,
                            input_length=input_shape,
                            trainable=True,
                            embeddings_regularizer=l2(0.0005),
                            name='ebd_rnn_'+col)(cur_input)
        x = SpatialDropout1D(0.5)(embed_layer)
        x = Bidirectional(CuDNNGRU(25, return_sequences=True))(x)
        x = Conv1D(25, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
        x_aveP = GlobalAveragePooling1D()(x)
        x_maxP = GlobalMaxPooling1D()(x)
        x = concatenate([x_aveP,x_maxP])
        input_list.append(cur_input)
        concat_list.append(x)

    if len(numu_cols) > 0:
        print('add numu...')
        nu_shape = len(numu_cols)
        cur_input = Input(shape=(nu_shape, ),name = prefix_input_nu)
        x = BatchNormalization()(cur_input)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        input_list.append(cur_input)
        numu_list.append(x)
       
    if len(concat_list) > 1:
        x = concatenate(concat_list)
#     x = BatchNormalization()(x)
    

    if len(numu_list)>0:
        x = concatenate([x]+numu_list)
#         x = BatchNormalization()(x)
        
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)

    preds = Dense(1, activation="sigmoid")(x)
    model = Model(input_list, preds)
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])    
    return model

def train_each_fold(input_train_dict,input_val_dict,y_train,y_val,cols,doc_col=[],tolerance=30,train_batch_size=5000):
    model = get_nn_model(cols,doc_col)
    cur_to = 0
    best_roc = None
    best_weights = None
    count = 0
    base_lr = 0.001
    while True:
        model.fit(input_train_dict, y_train, 
                  batch_size=train_batch_size, 
                  epochs=1,
                  verbose=1,
                  shuffle=True,
                  )
        preds = model.predict(input_val_dict,5000,verbose=1)
        roc = roc_auc_score(y_val,preds)
        print(roc)
        if best_roc is None:
            best_roc = roc
            best_weights = model.get_weights()
        else:
            if best_roc < roc:
                best_roc = roc
                best_weights = model.get_weights()
                cur_to = 0
            else:
                cur_to +=1
        if cur_to == tolerance:
            break
        print('best roc is: {}'.format(best_roc))
        print('remainning trial is: {}/{}'.format(cur_to,tolerance))
        print('total epoch trained: {}'.format(count))
        count+=1
    model.set_weights(best_weights)
    return model
    

def nn_K_fold(train_fold_dict,
              val_fold_dict,
              train_fold_y,
              val_fold_y,
              val_index_list,
              train_df,
              pred_col_name = 'predicted_score',
              nondoc_cols=[],
              doc_cols=[],
              tolerance=30,
              train_batch_size=5000,
              preds_batch=5000):
    """
    train_fold_dict: format, key - foldNum, value - nn input 
    train_fold_y: label for train fold, fotmat, key -foldNum, value - label
    val_fold_dict: format, key - foldNum, value - nn input 
    val_fold_y: label for val fold, fotmat, key -foldNum, value - label
    test_input_dict: format, key - foldNum, value - nn input 
    holdout_input_dict: Noneable, if none, not using holdout
    holdout_y: Noneable, label for holdout
    cols: all cols used to do ebd
    doc_cols, cols that are used to do rnn
    val_index_list: cv, going to predict and generate oof
    holdout_index_list: going to predict on each fold
    train_df: 'dataframe which only has id columns, which will be used to store oof prediction'
    test_df: 'dataframe which only has id columns, which will be used to store test prediction'
    """
    train_df = train_df.copy()
    n_fold = len(train_fold_dict)
    val_score_list = []
    train_score_list = []
    cv_score_list = []
    train_df[pred_col_name] = np.nan
    
    for fold in range(n_fold):
        print('start fold {}...'.format(fold))
        model = train_each_fold(train_fold_dict[fold],
                                val_fold_dict[fold],
                                train_fold_y[fold],
                                val_fold_y[fold],
                                nondoc_cols,
                                doc_cols,
                                tolerance=tolerance,
                                train_batch_size=train_batch_size)
        train_preds =  model.predict(train_fold_dict[fold],preds_batch,verbose=1)
        val_preds = model.predict(val_fold_dict[fold],preds_batch,verbose=1)
        train_roc = roc_auc_score(train_fold_y[fold],train_preds)
        val_roc = roc_auc_score(val_fold_y[fold],val_preds)
        train_df.loc[val_index_list[fold],pred_col_name] = val_preds
        val_score_list.append(val_roc)
        train_score_list.append(train_roc)
        print('Fold {} finish! val roc: {}.'.format(fold,val_roc))
        del model
        gc.collect()
        time.sleep(5)
            
    print('finish training... calculating evl matrix')
    cv_score_mean = np.mean(val_score_list)
    train_score_mean = np.mean(train_score_list)
    print('cv mean is: {}'.format(cv_score_mean))
    return train_df,cv_score_mean,train_score_mean
    



In [3]:
init = True
select_size = 40024
index_col = 'ori_index'
target_col = 'target'
ignore_columns = ['instance_id','time','click','ori_index','target'] + ['creative_is_js', 'creative_is_voicead', 'app_paid']
while True:
    if init:
        train = pd.read_pickle(FILE.train_ori.value)
        print('train shape is: {}'.format(train.shape))
        train['ori_index'] = train.index
        try:
            selected_index = pickle.load(open('../../data/original/adversarial_ho_index.pickle','rb'))
            train = train.loc[selected_index].copy()
            print('loading old index successfully!')
        except:
            print('no old index found!')
        train[target_col] = 0
        test = pd.read_pickle(FILE.test_ori.value)
        test[target_col] = 1
        print('test shape is: {}'.format(test.shape))
        train_length = len(train)
        X_shiyi = pd.read_pickle(FILE.shiyi_fillna_ori.value)
        X = pd.concat([train,test],sort=False)
        init = False  
    X = pd.concat([train,test],sort=False)
    X.reset_index(inplace=True)
    print(X.shape)
    X = X.merge(X_shiyi[['time_hour','instance_id']],how='inner',on='instance_id')
    need_process_col = list(set(X.columns) - set(ignore_columns))
    doc_col = ['user_tags','model']
    non_doc_col = [f for f in need_process_col if f not in doc_col]
    X_ = X[need_process_col].copy()
    X_doc = X[doc_col].copy()
    for col in tqdm(non_doc_col):
        X_[col] = le.fit_transform(X_[col].astype(str))
        X_[col] = col + '_'+X_[col].astype(str)
    
    for col in tqdm(doc_col):
        X_doc[col] = X_doc[col].astype(str)
        
    print('start preparing folds')
    num_folds = 3
    train_index_list = []
    val_index_list = []
    folds = StratifiedKFold(n_splits= num_folds, shuffle=True)
    for t,v in folds.split(X,X[target_col]):
        train_index_list.append(X.iloc[t].index)
        val_index_list.append(X.iloc[v].index)
    train_fold_y = {}
    val_fold_y = {}

    for fold in range(num_folds):
        train_fold_y[fold] = X.loc[train_index_list[fold],target_col].values
        val_fold_y[fold] = X.loc[val_index_list[fold],target_col].values
        
    print('start tokenizer')
    info_dict = {}
    train_all_dict = {}
    train_fold_dict = {}
    val_fold_dict = {}
    maxlen = 1
    prefix_input_nonDoc = 'input_'
    prefix_input_Doc = 'input_rnn_'
    for col in tqdm(non_doc_col):
        maxlen = 1
        tok=text.Tokenizer(num_words=X_[col].nunique(),lower=False,filters='@')
        tok.fit_on_texts(list(X_[col]))
        info_dict.update({prefix_input_nonDoc+col:{'tok':tok}})
        t = tok.texts_to_sequences(list(X_[col].values))
        train_all_dict[prefix_input_nonDoc+col] = sequence.pad_sequences(t,maxlen=maxlen)


        for fold in range(num_folds):
            if train_fold_dict.get(fold) is None:
                train_fold_dict[fold] = {}
                val_fold_dict[fold] = {}
            train_fold_dict[fold].update({prefix_input_nonDoc+col: train_all_dict[prefix_input_nonDoc+col][list(train_index_list[fold])]})
            val_fold_dict[fold].update({prefix_input_nonDoc+col:train_all_dict[prefix_input_nonDoc+col][list(val_index_list[fold])]})

    sequence_size_dict = {}
    
    for col in tqdm(doc_col):
        if col == 'user_tags':
            maxlen = 50
            tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False, filters=',')

        elif col == 'model':
            maxlen = 15
            tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False)
        else:
            maxlen = 15
            tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False)
        tok.fit_on_texts(list(X_doc[col]))
        info_dict.update({prefix_input_Doc+col:{'tok':tok}})
        sequence_size_dict[col] = maxlen
        t = tok.texts_to_sequences(list(X_doc[col].values))
        train_all_dict[prefix_input_Doc+col] = sequence.pad_sequences(t,maxlen=maxlen)

        for fold in range(num_folds):
            if train_fold_dict.get(fold) is None:
                train_fold_dict[fold] = {}
                val_fold_dict[fold] = {}
            train_fold_dict[fold].update({prefix_input_Doc+col: train_all_dict[prefix_input_Doc+col][list(train_index_list[fold])]})
            val_fold_dict[fold].update({prefix_input_Doc+col:train_all_dict[prefix_input_Doc+col][list(val_index_list[fold])]})
            
    train_df = X[[target_col,index_col]].copy()
    train_batch_size = int(len(train) * 0.6 / 120)
    if train_batch_size > 5000:
        train_batch_size=5000
    if train_batch_size < 500:
        train_batch_size=500
    
    train_save,cv_,ta_ = nn_K_fold(train_fold_dict,
                                          val_fold_dict,
                                          train_fold_y,
                                          val_fold_y,
                                          val_index_list,
                                          train_df,
                                          pred_col_name = 'predicted_score',
                                          nondoc_cols=non_doc_col,
                                          doc_cols=doc_col,
                                          tolerance=10,
                                           train_batch_size=train_batch_size,
                                          preds_batch=5000)
    
    
    
    real_train = train_save[train_save[target_col]==0].copy()
    real_train_length = len(real_train)
    ratio = 0.2
    real_train = real_train.sort_values('predicted_score').iloc[min(int(real_train_length*ratio),real_train_length-select_size):]
    selected_index = list(real_train[index_col].astype(int).values)
    pickle.dump(selected_index,open('../../data/original/adversarial_ho_index.pickle','wb'))
    train = train.loc[selected_index].copy()
    if len(train) == select_size:
        print('holdout selected! Done!')
        break
    

train shape is: (1001650, 35)
loading old index successfully!
test shape is: (40024, 35)
(84079, 38)


  0%|          | 0/29 [00:00<?, ?it/s]

start preparing folds
start tokenizer


100%|██████████| 2/2 [00:03<00:00,  1.81s/it]


start fold 0...
Epoch 1/1
28027/28027 [==============================] - 1s 18us/step
0.6021888096537047
best roc is: 0.6021888096537047
remainning trial is: 0/10
total epoch trained: 0
Epoch 1/1
28027/28027 [==============================] - 0s 8us/step
0.6100344709544516
best roc is: 0.6100344709544516
remainning trial is: 0/10
total epoch trained: 1
Epoch 1/1
28027/28027 [==============================] - 0s 8us/step
0.6117695944010244
best roc is: 0.6117695944010244
remainning trial is: 0/10
total epoch trained: 2
Epoch 1/1
28027/28027 [==============================] - 0s 8us/step
0.5874046655169542
best roc is: 0.6117695944010244
remainning trial is: 1/10
total epoch trained: 3
Epoch 1/1
28027/28027 [==============================] - 0s 8us/step
0.5905498147348248
best roc is: 0.6117695944010244
remainning trial is: 2/10
total epoch trained: 4
Epoch 1/1
28027/28027 [==============================] - 0s 8us/step
0.5864813815861365
best roc is: 0.6117695944010244
remainning trial i

28026/28026 [==============================] - 0s 8us/step
0.5754411846487555
best roc is: 0.5990562576671632
remainning trial is: 5/10
total epoch trained: 6
Epoch 1/1
28026/28026 [==============================] - 0s 8us/step
0.5752793420596232
best roc is: 0.5990562576671632
remainning trial is: 6/10
total epoch trained: 7
Epoch 1/1
28026/28026 [==============================] - 0s 8us/step
0.571029273081155
best roc is: 0.5990562576671632
remainning trial is: 7/10
total epoch trained: 8
Epoch 1/1
28026/28026 [==============================] - 0s 8us/step
0.5737414597433851
best roc is: 0.5990562576671632
remainning trial is: 8/10
total epoch trained: 9
Epoch 1/1
28026/28026 [==============================] - 0s 8us/step
0.5734773547089893
best roc is: 0.5990562576671632
remainning trial is: 9/10
total epoch trained: 10
Epoch 1/1
28026/28026 [==============================] - 0s 8us/step
0.5787320707345065
28026/28026 [==============================] - 0s 8us/step
Fold 2 finish! val

In [4]:
# ss = train.copy()

In [8]:
train = pd.read_pickle(FILE.train_ori.value)
selected_index = pickle.load(open('../../data/original/adversarial_ho_index.pickle','rb'))
train_index = list(set(train.index)-set(selected_index))
print(len(train_index))
pickle.dump(train_index,open('../../data/original/adversarial_train_index.pickle','wb'))

961626


In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import pickle

In [2]:
train = pd.read_pickle(FILE.train_ori.value)
selected_index = pickle.load(open('../../data/original/adversarial_ho_index.pickle','rb'))
ho = train.loc[selected_index].copy()

In [4]:
np.sum(ho['user_tags'].isnull())/len(ho)

0.3330501698980612

In [4]:
ho['model']

97005            HLTEM800
137567             iPhone
746511           Redmi-3S
387509           OPPO A73
591587           PRA-AL00
236263          vivo Y55A
315273           vivo-Y51
484878          YU-FLY-F9
605305          vivo-Y85A
306155           OPPO A33
825854           OPPO-R9s
13906           OPPO-A37m
433249           vivo-Y35
259827            Le-X528
38144       vivo-X9s-Plus
609986            MI-4LTE
206535           BAC-AL00
161984           vivo Y31
731138           CAM-AL00
936239           PLE-703L
371638            MI 4LTE
153575         KINGSUN-F9
496798                U20
136417          iPhone7,2
37745                MI-8
634704          SUGAR+F11
65923           boway-U15
169856           vivo-Y51
603516          OPPO R7sm
444599           vivoX20A
               ...       
37067              M651CY
20931             vivo X9
9338             GT-I9508
26137            COL-AL10
17804                  15
1925       iPhone 6s Plus
2837            vivo Y66L
27851       